In [31]:
import pandas as pd
import numpy as np
import re
from datetime import datetime as dt
from datetime import timedelta
import json


# Data Load and Combine:

In [75]:
#Load Fighters:
fighters = pd.read_csv('Data/fighter.csv', index_col = 0)
fighters.head(5)
fighters.reset_index(drop = True, inplace = True)

In [76]:
#Load fight_details:
data = []

for i in np.arange(1, 7):
    data.append(pd.read_csv('Data/fight_details_batch{}.csv'.format(i), index_col = 0))

In [77]:
df = pd.DataFrame()

for item in data:
    df = pd.concat([df, item])
    
df.shape

df.reset_index(drop = True, inplace = True)

In [78]:
df.columns

Index(['Fighter', 'KD', 'Sig. str. %', 'Td %', 'Sub. att', 'Pass', 'Rev.',
       'Sig. str. Hits', 'Sig. str. Attempts', 'Total str. Hits',
       'Total str. Attempts', 'Td Hits', 'Td Attempts', 'Head Hits',
       'Head Attempts', 'Body Hits', 'Body Attempts', 'Leg Hits',
       'Leg Attempts', 'Distance Hits', 'Distance Attempts', 'Clinch Hits',
       'Clinch Attempts', 'Ground Hits', 'Ground Attempts', 'Fighter.1',
       'KD.1', 'Sig. str. %.1', 'Td %.1', 'Sub. att.1', 'Pass.1', 'Rev..1',
       'Sig. str. Hits.1', 'Sig. str. Attempts.1', 'Total str. Hits.1',
       'Total str. Attempts.1', 'Td Hits.1', 'Td Attempts.1', 'Head Hits.1',
       'Head Attempts.1', 'Body Hits.1', 'Body Attempts.1', 'Leg Hits.1',
       'Leg Attempts.1', 'Distance Hits.1', 'Distance Attempts.1',
       'Clinch Hits.1', 'Clinch Attempts.1', 'Ground Hits.1',
       'Ground Attempts.1', 'rounds', 'time', 'format', 'referee', 'url'],
      dtype='object')

# Clean Event Details Data:

### Find Null Values:

In [79]:
drop_index = df[df['KD'] == 99999].index
df.drop(drop_index, axis = 0, inplace = True)

### Change Data Formats:

In [80]:
df.dtypes

Fighter                  object
KD                        int64
Sig. str. %              object
Td %                     object
Sub. att                  int64
Pass                      int64
Rev.                      int64
Sig. str. Hits            int64
Sig. str. Attempts        int64
Total str. Hits           int64
Total str. Attempts       int64
Td Hits                   int64
Td Attempts               int64
Head Hits                 int64
Head Attempts             int64
Body Hits                 int64
Body Attempts             int64
Leg Hits                  int64
Leg Attempts              int64
Distance Hits             int64
Distance Attempts         int64
Clinch Hits               int64
Clinch Attempts           int64
Ground Hits               int64
Ground Attempts           int64
Fighter.1                object
KD.1                      int64
Sig. str. %.1            object
Td %.1                   object
Sub. att.1                int64
Pass.1                    int64
Rev..1  

In [81]:
df.columns

Index(['Fighter', 'KD', 'Sig. str. %', 'Td %', 'Sub. att', 'Pass', 'Rev.',
       'Sig. str. Hits', 'Sig. str. Attempts', 'Total str. Hits',
       'Total str. Attempts', 'Td Hits', 'Td Attempts', 'Head Hits',
       'Head Attempts', 'Body Hits', 'Body Attempts', 'Leg Hits',
       'Leg Attempts', 'Distance Hits', 'Distance Attempts', 'Clinch Hits',
       'Clinch Attempts', 'Ground Hits', 'Ground Attempts', 'Fighter.1',
       'KD.1', 'Sig. str. %.1', 'Td %.1', 'Sub. att.1', 'Pass.1', 'Rev..1',
       'Sig. str. Hits.1', 'Sig. str. Attempts.1', 'Total str. Hits.1',
       'Total str. Attempts.1', 'Td Hits.1', 'Td Attempts.1', 'Head Hits.1',
       'Head Attempts.1', 'Body Hits.1', 'Body Attempts.1', 'Leg Hits.1',
       'Leg Attempts.1', 'Distance Hits.1', 'Distance Attempts.1',
       'Clinch Hits.1', 'Clinch Attempts.1', 'Ground Hits.1',
       'Ground Attempts.1', 'rounds', 'time', 'format', 'referee', 'url'],
      dtype='object')

In [82]:
df['Sig. str. %'] = df['Sig. str. %'].apply(lambda x: re.sub('%', '', x))
df['Td %'] = df['Td %'].apply(lambda x: re.sub('%', '', x))

df['Sig. str. %.1'] = df['Sig. str. %.1'].apply(lambda x: re.sub('%', '', x))
df['Td %.1'] = df['Td %.1'].apply(lambda x: re.sub('%', '', x))

In [83]:
df['Sig. str. %'] = df['Sig. str. %'].astype(float)
df['Sig. str. %'] = df['Sig. str. %'] / 100

df['Td %'] = df['Td %'].astype(float)
df['Td %'] = df['Td %'] / 100

df['Sig. str. %.1'] = df['Sig. str. %.1'].astype(float)
df['Sig. str. %.1'] = df['Sig. str. %.1'] / 100

df['Td %.1'] = df['Td %.1'].astype(float)
df['Td %.1'] = df['Td %.1'] / 100

# Combine with main_event:

In [84]:
event_data = pd.read_csv('Data/event_level_data.csv', index_col = 0)
event_data.drop(drop_index, axis = 0, inplace = True)

In [85]:
combined_df = pd.concat([event_data, df], axis = 1)
combined_df.head(5)

,Winner,R_fighter,B_fighter,R_STR,B_STR,R_TD,B_TD,R_SUB,B_SUB,R_PASS,...,Distance Attempts.1,Clinch Hits.1,Clinch Attempts.1,Ground Hits.1,Ground Attempts.1,rounds,time,format,referee,url
0,win,Charles Oliveira,Kevin Lee,43,41,0,2,2,0,0,...,56,2,2,6,7,3,00:00:28,5 Rnd (5-5-5-5-5),Mike Beltran,http://ufcstats.com/fight-details/e0b323dae5bf...
1,win,Gilbert Burns,Demian Maia,13,4,0,2,0,0,0,...,7,0,0,8,9,1,00:02:34,3 Rnd (5-5-5),Osiris Maia,http://ufcstats.com/fight-details/e0b323dae5bf...
2,win,Renato Moicano,Damir Hadzovic,1,1,1,0,1,0,1,...,5,0,0,0,0,1,00:00:44,3 Rnd (5-5-5),Eduardo Herdy,http://ufcstats.com/fight-details/e0b323dae5bf...
3,win,Nikita Krylov,Johnny Walker,45,37,3,0,0,0,4,...,18,2,3,33,37,3,00:05:00,3 Rnd (5-5-5),Mike Beltran,http://ufcstats.com/fight-details/e0b323dae5bf...
4,win,Francisco Trinaldo,John Makdessi,55,67,0,0,0,0,0,...,121,2,2,0,0,3,00:05:00,3 Rnd (5-5-5),Osiris Maia,http://ufcstats.com/fight-details/e0b323dae5bf...


In [19]:
combined_df.isna().any()

Winner       False
R_fighter    False
B_fighter    False
R_STR        False
B_STR        False
             ...  
rounds       False
time         False
format       False
referee       True
url          False
Length: 76, dtype: bool

In [340]:
#Drop redundant data:
drop_columns = ['R_STR', 'B_STR', 'R_TD', 'B_TD',
       'R_SUB', 'B_SUB', 'R_PASS', 'B_PASS', 'TIME', 'ROUND', 'link']
combined_df.drop(drop_columns, axis = 1, inplace = True)

# Feature Engineering:

### Winner:

In [341]:
winner = []
for index, row in combined_df.iterrows():
    
    if row['Winner'] != 'win':
        winner.append(99999)
    
    elif (row['R_fighter'] == row['Fighter']):
        winner.append(0)
    else:
        winner.append(1)

In [342]:
combined_df['label'] = winner

In [343]:
combined_df.head(5)

,Winner,R_fighter,B_fighter,WEIGHT_CLASS,METHOD,DETAIL,title_bout,date,location,attendance,Fighter,KD,Sig. str. %,Td %,Sub. att,Pass,Rev.,Sig. str. Hits,Sig. str. Attempts,Total str. Hits,Total str. Attempts,Td Hits,Td Attempts,Head Hits,Head Attempts,Body Hits,Body Attempts,Leg Hits,Leg Attempts,Distance Hits,Distance Attempts,Clinch Hits,Clinch Attempts,Ground Hits,Ground Attempts,Fighter.1,KD.1,Sig. str. %.1,Td %.1,Sub. att.1,Pass.1,Rev..1,Sig. str. Hits.1,Sig. str. Attempts.1,Total str. Hits.1,Total str. Attempts.1,Td Hits.1,Td Attempts.1,Head Hits.1,Head Attempts.1,Body Hits.1,Body Attempts.1,Leg Hits.1,Leg Attempts.1,Distance Hits.1,Distance Attempts.1,Clinch Hits.1,Clinch Attempts.1,Ground Hits.1,Ground Attempts.1,rounds,time,format,referee,url,label
0,win,Charles Oliveira,Kevin Lee,Lightweight,SUB,Guillotine Choke,0.0,14-03-2020,"Brasilia, Distrito Federal, Brazil",0.0,Kevin Lee,0,0.51,0.66,0,2,1,41,80,61,100,2,3,35,74,6,6,0,0,31,65,0,0,10,15,Charles Oliveira,0,0.66,0.00,2,0,1,43,65,51,73,0,1,28,49,9,10,6,6,35,56,2,2,6,7,3,00:00:28,5 Rnd (5-5-5-5-5),Mike Beltran,http://ufcstats.com/fight-details/e0b323dae5bf...,1
1,win,Gilbert Burns,Demian Maia,Welterweight,KO/TKO,Punch,0.0,14-03-2020,"Brasilia, Distrito Federal, Brazil",0.0,Demian Maia,0,0.57,1.00,0,1,0,4,7,4,7,2,2,4,7,0,0,0,0,4,7,0,0,0,0,Gilbert Burns,1,0.81,0.00,0,0,0,13,16,14,17,0,0,10,13,1,1,2,2,5,7,0,0,8,9,1,00:02:34,3 Rnd (5-5-5),Osiris Maia,http://ufcstats.com/fight-details/e0b323dae5bf...,1
2,win,Renato Moicano,Damir Hadzovic,Lightweight,SUB,Rear Naked Choke,0.0,14-03-2020,"Brasilia, Distrito Federal, Brazil",0.0,Renato Moicano,0,0.50,1.00,1,1,0,1,2,4,5,1,1,1,2,0,0,0,0,1,2,0,0,0,0,Damir Hadzovic,0,0.20,0.00,0,0,0,1,5,1,5,0,0,0,4,0,0,1,1,1,5,0,0,0,0,1,00:00:44,3 Rnd (5-5-5),Eduardo Herdy,http://ufcstats.com/fight-details/e0b323dae5bf...,0
3,win,Nikita Krylov,Johnny Walker,Light Heavyweight,U-DEC,NaN,0.0,14-03-2020,"Brasilia, Distrito Federal, Brazil",0.0,Johnny Walker,0,0.74,0.00,0,2,1,37,50,91,112,0,0,33,45,2,3,2,2,10,14,8,9,19,27,Nikita Krylov,0,0.77,0.37,0,4,1,45,58,128,148,3,8,38,49,6,7,1,2,10,18,2,3,33,37,3,00:05:00,3 Rnd (5-5-5),Mike Beltran,http://ufcstats.com/fight-details/e0b323dae5bf...,1
4,win,Francisco Trinaldo,John Makdessi,Lightweight,U-DEC,NaN,0.0,14-03-2020,"Brasilia, Distrito Federal, Brazil",0.0,Francisco Trinaldo,0,0.43,0.00,0,0,0,55,126,55,126,0,1,34,100,17,22,4,4,53,124,2,2,0,0,John Makdessi,0,0.54,0.00,0,0,0,67,123,68,124,0,0,15,54,16,26,36,43,65,121,2,2,0,0,3,00:05:00,3 Rnd (5-5-5),Osiris Maia,http://ufcstats.com/fight-details/e0b323dae5bf...,0


### Match Time:

In [344]:
#Reformat match format:

round_times = combined_df['format'].apply(lambda x: re.search(r'\(.*\)', x))
round_times = round_times.apply(lambda x: x.group().strip('()').split('-') if x != None else [1])
round_times = round_times.apply(lambda x: np.array(x, dtype = 'float'))
round_times[0:5]

0    [5.0, 5.0, 5.0, 5.0, 5.0]
1              [5.0, 5.0, 5.0]
2              [5.0, 5.0, 5.0]
3              [5.0, 5.0, 5.0]
4              [5.0, 5.0, 5.0]
Name: format, dtype: object

In [345]:
#Returns time in minutes given datetime object:
def to_minutes(time):
    
    set_time = dt.strptime(time, '%H:%M:%S')
    
    return set_time.minute + set_time.second / 60

In [347]:
#Calculate total match time
round_dot = []

for num_rounds, time, item in zip(combined_df['rounds'], combined_df['time'], round_times):
    
    minutes = to_minutes(time)
    
    round_indicator = np.zeros(len(item))
    round_indicator[0: num_rounds-1] = 1
    
    round_time = round(np.dot(round_indicator, item) + minutes, 2)
    
    round_dot.append(round_time)
    


In [348]:
combined_df['match_time'] = round_dot

In [349]:
combined_df.reset_index(inplace = True)
combined_df

,index,Winner,R_fighter,B_fighter,WEIGHT_CLASS,METHOD,DETAIL,title_bout,date,location,attendance,Fighter,KD,Sig. str. %,Td %,Sub. att,Pass,Rev.,Sig. str. Hits,Sig. str. Attempts,Total str. Hits,Total str. Attempts,Td Hits,Td Attempts,Head Hits,Head Attempts,Body Hits,Body Attempts,Leg Hits,Leg Attempts,Distance Hits,Distance Attempts,Clinch Hits,Clinch Attempts,Ground Hits,Ground Attempts,Fighter.1,KD.1,Sig. str. %.1,Td %.1,Sub. att.1,Pass.1,Rev..1,Sig. str. Hits.1,Sig. str. Attempts.1,Total str. Hits.1,Total str. Attempts.1,Td Hits.1,Td Attempts.1,Head Hits.1,Head Attempts.1,Body Hits.1,Body Attempts.1,Leg Hits.1,Leg Attempts.1,Distance Hits.1,Distance Attempts.1,Clinch Hits.1,Clinch Attempts.1,Ground Hits.1,Ground Attempts.1,rounds,time,format,referee,url,label,match_time
0,0,win,Charles Oliveira,Kevin Lee,Lightweight,SUB,Guillotine Choke,0.0,14-03-2020,"Brasilia, Distrito Federal, Brazil",0.0,Kevin Lee,0,0.51,0.66,0,2,1,41,80,61,100,2,3,35,74,6,6,0,0,31,65,0,0,10,15,Charles Oliveira,0,0.66,0.00,2,0,1,43,65,51,73,0,1,28,49,9,10,6,6,35,56,2,2,6,7,3,00:00:28,5 Rnd (5-5-5-5-5),Mike Beltran,http://ufcstats.com/fight-details/e0b323dae5bf...,1,10.47
1,1,win,Gilbert Burns,Demian Maia,Welterweight,KO/TKO,Punch,0.0,14-03-2020,"Brasilia, Distrito Federal, Brazil",0.0,Demian Maia,0,0.57,1.00,0,1,0,4,7,4,7,2,2,4,7,0,0,0,0,4,7,0,0,0,0,Gilbert Burns,1,0.81,0.00,0,0,0,13,16,14,17,0,0,10,13,1,1,2,2,5,7,0,0,8,9,1,00:02:34,3 Rnd (5-5-5),Osiris Maia,http://ufcstats.com/fight-details/e0b323dae5bf...,1,2.57
2,2,win,Renato Moicano,Damir Hadzovic,Lightweight,SUB,Rear Naked Choke,0.0,14-03-2020,"Brasilia, Distrito Federal, Brazil",0.0,Renato Moicano,0,0.50,1.00,1,1,0,1,2,4,5,1,1,1,2,0,0,0,0,1,2,0,0,0,0,Damir Hadzovic,0,0.20,0.00,0,0,0,1,5,1,5,0,0,0,4,0,0,1,1,1,5,0,0,0,0,1,00:00:44,3 Rnd (5-5-5),Eduardo Herdy,http://ufcstats.com/fight-details/e0b323dae5bf...,0,0.73
3,3,win,Nikita Krylov,Johnny Walker,Light Heavyweight,U-DEC,NaN,0.0,14-03-2020,"Brasilia, Distrito Federal, Brazil",0.0,Johnny Walker,0,0.74,0.00,0,2,1,37,50,91,112,0,0,33,45,2,3,2,2,10,14,8,9,19,27,Nikita Krylov,0,0.77,0.37,0,4,1,45,58,128,148,3,8,38,49,6,7,1,2,10,18,2,3,33,37,3,00:05:00,3 Rnd (5-5-5),Mike Beltran,http://ufcstats.com/fight-details/e0b323dae5bf...,1,15.00
4,4,win,Francisco Trinaldo,John Makdessi,Lightweight,U-DEC,NaN,0.0,14-03-2020,"Brasilia, Distrito Federal, Brazil",0.0,Francisco Trinaldo,0,0.43,0.00,0,0,0,55,126,55,126,0,1,34,100,17,22,4,4,53,124,2,2,0,0,John Makdessi,0,0.54,0.00,0,0,0,67,123,68,124,0,0,15,54,16,26,36,43,65,121,2,2,0,0,3,00:05:00,3 Rnd (5-5-5),Osiris Maia,http://ufcstats.com/fight-details/e0b323dae5bf...,0,15.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5533,5559,win,Gerard Gordeau,Kevin Rosier,Open Weight,KO/TKO,NaN,0.0,12-11-1993,"Denver, Colorado, USA",2800.0,Gerard Gordeau,1,0.64,0.00,0,0,0,11,17,11,17,0,0,7,13,1,1,3,3,5,8,0,0,6,9,Kevin Rosier,0,0.00,0.00,0,0,0,0,3,0,3,0,0,0,1,0,1,0,1,0,3,0,0,0,0,1,00:00:59,No Time Limit,Joao Alberto Barreto,http://ufcstats.com/fight-details/ac7ca2ec38b9...,0,0.98
5534,5560,win,Ken Shamrock,Patrick Smith,Open Weight,SUB,Heel Hook,0.0,12-11-1993,"Denver, Colorado, USA",2800.0,Ken Shamrock,0,1.00,0.50,2,0,0,1,1,4,4,1,2,1,1,0,0,0,0,0,0,0,0,1,1,Patrick Smith,0,0.50,0.00,0,0,0,4,8,16,20,0,0,1,4,1,1,2,3,1,1,1,1,2,6,1,00:01:49,No Time Limit,Joao Alberto Barreto,http://ufcstats.com/fight-details/46acd54cc0c9...,0,1.82
5535,5561,win,Royce Gracie,Art Jimmerson,Open Weight,SUB,Other,0.0,12-11-1993,"Denver, Colorado, USA",2800.0,Royce Gracie,0,0.00,1.00,0,2,0,0,3,4,7,1,1,0,1,0,0,0,2,0,3,0,0,0,0,Art Jimmerson,0,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,00:02:18,No Time Limit,Joao Alberto Barreto,http://ufcstats.com/fight-details/cecdc0da5842...,0,2.30
5536,5562,win,Kevin Rosier,Zane Fra

# Combine Fighter Information:

In [350]:
player_columns = ['DOB', 'Height', 'Reach', 'Stance', 'Weight',
                 'DOB.1', 'Height.1', 'Reach.1', 'Stance.1', 'Weight.1',]

In [351]:
R_B_combined = pd.DataFrame()

#Given 2 players in a match, retrieve their corresponding personal attributes (height, weight, reach, etc.)

for index, row in combined_df.iterrows():
    
    R_fighter = row['Fighter']
    B_fighter = row['Fighter.1']
    
    R_stats = fighters[fighters['Name'] == R_fighter].copy()
    B_stats = fighters[fighters['Name'] == B_fighter].copy()
    
    R_stats.reset_index(drop = True, inplace = True)
    B_stats.reset_index(drop = True, inplace = True)
    
    combined = pd.concat([R_stats, B_stats], axis = 1)
    R_B_combined = pd.concat([R_B_combined, combined], axis = 0)
    

In [353]:
R_B_combined.drop('Name', axis = 1, inplace = True)
R_B_combined.columns = player_columns
R_B_combined.reset_index(drop = True, inplace = True)

In [354]:
data = pd.concat([combined_df, R_B_combined], axis = 1)
data.shape

(5538, 78)

In [355]:
data.to_csv('fights.csv')

# Navigating the Data:

In [356]:
df = pd.read_csv('fights.csv', index_col = 0)
df.head(5)

,index,Winner,R_fighter,B_fighter,WEIGHT_CLASS,METHOD,DETAIL,title_bout,date,location,attendance,Fighter,KD,Sig. str. %,Td %,Sub. att,Pass,Rev.,Sig. str. Hits,Sig. str. Attempts,Total str. Hits,Total str. Attempts,Td Hits,Td Attempts,Head Hits,Head Attempts,Body Hits,Body Attempts,Leg Hits,Leg Attempts,Distance Hits,Distance Attempts,Clinch Hits,Clinch Attempts,Ground Hits,Ground Attempts,Fighter.1,KD.1,Sig. str. %.1,Td %.1,Sub. att.1,Pass.1,Rev..1,Sig. str. Hits.1,Sig. str. Attempts.1,Total str. Hits.1,Total str. Attempts.1,Td Hits.1,Td Attempts.1,Head Hits.1,Head Attempts.1,Body Hits.1,Body Attempts.1,Leg Hits.1,Leg Attempts.1,Distance Hits.1,Distance Attempts.1,Clinch Hits.1,Clinch Attempts.1,Ground Hits.1,Ground Attempts.1,rounds,time,format,referee,url,label,match_time,DOB,Height,Reach,Stance,Weight,DOB.1,Height.1,Reach.1,Stance.1,Weight.1
0,0,win,Charles Oliveira,Kevin Lee,Lightweight,SUB,Guillotine Choke,0.0,14-03-2020,"Brasilia, Distrito Federal, Brazil",0.0,Kevin Lee,0,0.51,0.66,0,2,1,41,80,61,100,2,3,35,74,6,6,0,0,31,65,0,0,10,15,Charles Oliveira,0,0.66,0.00,2,0,1,43,65,51,73,0,1,28,49,9,10,6,6,35,56,2,2,6,7,3,00:00:28,5 Rnd (5-5-5-5-5),Mike Beltran,http://ufcstats.com/fight-details/e0b323dae5bf...,1,10.47,04-09-1992,69,77,Orthodox,170,17-10-1989,61,74,Orthodox,155
1,1,win,Gilbert Burns,Demian Maia,Welterweight,KO/TKO,Punch,0.0,14-03-2020,"Brasilia, Distrito Federal, Brazil",0.0,Demian Maia,0,0.57,1.00,0,1,0,4,7,4,7,2,2,4,7,0,0,0,0,4,7,0,0,0,0,Gilbert Burns,1,0.81,0.00,0,0,0,13,16,14,17,0,0,10,13,1,1,2,2,5,7,0,0,8,9,1,00:02:34,3 Rnd (5-5-5),Osiris Maia,http://ufcstats.com/fight-details/e0b323dae5bf...,1,2.57,06-11-1977,73,72,Southpaw,170,20-07-1986,61,71,Orthodox,170
2,2,win,Renato Moicano,Damir Hadzovic,Lightweight,SUB,Rear Naked Choke,0.0,14-03-2020,"Brasilia, Distrito Federal, Brazil",0.0,Renato Moicano,0,0.50,1.00,1,1,0,1,2,4,5,1,1,1,2,0,0,0,0,1,2,0,0,0,0,Damir Hadzovic,0,0.20,0.00,0,0,0,1,5,1,5,0,0,0,4,0,0,1,1,1,5,0,0,0,0,1,00:00:44,3 Rnd (5-5-5),Eduardo Herdy,http://ufcstats.com/fight-details/e0b323dae5bf...,0,0.73,21-05-1989,61,72,Orthodox,155,08-08-1986,69,70,Orthodox,155
3,3,win,Nikita Krylov,Johnny Walker,Light Heavyweight,U-DEC,NaN,0.0,14-03-2020,"Brasilia, Distrito Federal, Brazil",0.0,Johnny Walker,0,0.74,0.00,0,2,1,37,50,91,112,0,0,33,45,2,3,2,2,10,14,8,9,19,27,Nikita Krylov,0,0.77,0.37,0,4,1,45,58,128,148,3,8,38,49,6,7,1,2,10,18,2,3,33,37,3,00:05:00,3 Rnd (5-5-5),Mike Beltran,http://ufcstats.com/fight-details/e0b323dae5bf...,1,15.00,30-03-1992,78,82,Orthodox,205,07-03-1992,75,77,Orthodox,205
4,4,win,Francisco Trinaldo,John Makdessi,Lightweight,U-DEC,NaN,0.0,14-03-2020,"Brasilia, Distrito Federal, Brazil",0.0,Francisco Trinaldo,0,0.43,0.00,0,0,0,55,126,55,126,0,1,34,100,17,22,4,4,53,124,2,2,0,0,John Makdessi,0,0.54,0.00,0,0,0,67,123,68,124,0,0,15,54,16,26,36,43,65,121,2,2,0,0,3,00:05:00,3 Rnd (5-5-5),Osiris Maia,http://ufcstats.com/fight-details/e0b323dae5bf...,0,15.00,24-08-1978,69,70,Southpaw,155,03-05-1985,68,68,Orthodox,155


In [357]:
#Relevant columns when breaking out data for specific fighters

fight_columns = ['Winner', 'R_fighter', 'Fighter', 'KD',
       'Sig. str. %', 'Td %', 'Sub. att', 'Pass', 'Rev.', 'Sig. str. Hits',
       'Sig. str. Attempts', 'Total str. Hits', 'Total str. Attempts',
       'Td Hits', 'Td Attempts', 'Head Hits', 'Head Attempts', 'Body Hits',
       'Body Attempts', 'Leg Hits', 'Leg Attempts', 'Distance Hits',
       'Distance Attempts', 'Clinch Hits', 'Clinch Attempts', 'Ground Hits',
       'Ground Attempts', 'DOB', 'Height', 'Reach',
       'Stance', 'Weight', 'date', 'match_time']

fight_columns_1 = ['Winner','R_fighter','Fighter.1', 'KD.1', 'Sig. str. %.1', 'Td %.1',
       'Sub. att.1', 'Pass.1', 'Rev..1', 'Sig. str. Hits.1',
       'Sig. str. Attempts.1', 'Total str. Hits.1', 'Total str. Attempts.1',
       'Td Hits.1', 'Td Attempts.1', 'Head Hits.1', 'Head Attempts.1',
       'Body Hits.1', 'Body Attempts.1', 'Leg Hits.1', 'Leg Attempts.1',
       'Distance Hits.1', 'Distance Attempts.1', 'Clinch Hits.1',
       'Clinch Attempts.1', 'Ground Hits.1', 'Ground Attempts.1', 'DOB.1', 'Height.1', 'Reach.1', 'Stance.1',
       'Weight.1', 'date', 'match_time']

In [358]:
def get_fight_attributes(index):
    
    temp = df.loc[index]
    return temp[['Fighter', 'Fighter.1', 'title_bout',
                'date', 'location', 'attendance', 'rounds', 'time', 'format', 'referee']]
    

def get_fights(fighter):
    
    data = df[(df['Fighter'] == fighter) | (df['Fighter.1'] == fighter)]
    
    return data


#get_personal returns fight statistics only fighter  
def get_personal(fights, fighter):
    
    a = fights[fights['Fighter'] == fighter]
    a1 = fights[fights['Fighter.1'] == fighter]
    
    a = a[fight_columns].copy()
    a1 = a1[fight_columns_1].copy()

    a1.columns = fight_columns  
    
    combined = pd.concat([a, a1], axis = 0)
    combined.sort_index(ascending = False, inplace = True)
    
    return combined
    

# Feature Engineering

In [523]:
def calc_age(personal):
    try:
        dob = dt.strptime(personal['DOB'].iloc[0], '%d-%m-%Y')
        match_dates = personal['date'].apply(lambda x: dt.strptime(x, '%d-%m-%Y'))

        ages = match_dates.apply(lambda x: round(((x - dob).days)/365,2))
    except:
        ages = pd.Series(float(0.0), index=personal.index)
        
    ages = ages.rename('age')

    return ages


def calc_WL(personal, shift = True):
        
    wins = personal['R_fighter'].apply(lambda x: x == personal['Fighter'].iloc[0])
    losses = personal['R_fighter'].apply(lambda x: x != personal['Fighter'].iloc[0])
    draws = pd.Series(0, index = wins.index)
    
    for idx in wins.index:
        if personal['Winner'].loc[idx] != 'win':
            draws[idx] = 1
            wins[idx] = False
            losses[idx] = False
    
    #Winning/Losing Streak Algorith:
    switch = []
    for item in np.arange(1, len(wins)):
        if wins.iloc[item] == wins.iloc[item-1]:
            switch.append(False)
        else:
            switch.append(True)
    
    if wins.iloc[0]:
        streak = [1]
    else:
        streak = [-1]

    for item, swap in zip(wins, switch):
        if swap:
            streak.append(1 * -np.sign(streak[-1]))
        else:
            streak.append(streak[-1] + np.sign(streak[-1])*1)

            
    streak = pd.Series(streak, index = wins.index)
    #End streak algorithm
        
    
    wl = pd.DataFrame([np.cumsum(losses), np.cumsum(wins), np.cumsum(draws)]).T
    wl = pd.concat([wl, streak], axis = 1)
    wl.columns = ['loss', 'win', 'draws', 'streak']
    
    if shift:
        wl = wl.shift(1)
    
    wl.iloc[0, :] = 0

    return wl



def calc_time(personal, shift = True):

    time = np.cumsum(personal['match_time'])
    initial = time.iloc[0]
    
    if shift:
        time = time.shift(1)
    
    time.iloc[0] = initial
    time = time.rename('cum_match_time')

    return time


def calc_stats(personal, shift = True):
    stat_columns = ['KD', 'Sub. att', 'Pass',
       'Rev.', 'Sig. str. Hits', 'Sig. str. Attempts', 'Total str. Hits',
       'Total str. Attempts', 'Td Hits', 'Td Attempts', 'Head Hits',
       'Head Attempts', 'Body Hits', 'Body Attempts', 'Leg Hits',
       'Leg Attempts', 'Distance Hits', 'Distance Attempts', 'Clinch Hits',
       'Clinch Attempts', 'Ground Hits', 'Ground Attempts']
    
    stats = personal[stat_columns]
    
    cumulative_columns = ['cum_{}'.format(item) for item in stat_columns]
    
    stats = np.cumsum(stats)
    initial = stats.iloc[0, :].copy()
    
    if shift:
        stats = stats.shift(1)
    
    stats.iloc[0, :] = initial
    stats.columns = cumulative_columns
    
    return stats


def get_calculated_personal(fighter, shift = True):
    
    if (len(df[df['Fighter'] == fighter]) == 0) & (len(df[df['Fighter.1'] == fighter]) == 0):
        return None
    
    
    personal = get_personal(df, fighter)
    
    ages = calc_age(personal)
    
    wl = calc_WL(personal, shift)
    round_times = calc_time(personal, shift)
    stats = calc_stats(personal, shift)
    
    final = pd.concat([personal, ages, wl, round_times, stats], axis = 1)
    
    return final



In [524]:
test_personal = get_personal(df, 'Dustin Poirier')
len(test_personal)
calc_WL(test_personal)

,loss,win,draws,streak
4033,0.0,0.0,0.0,0.0
3922,0.0,1.0,0.0,1.0
3784,0.0,2.0,0.0,2.0
3702,0.0,3.0,0.0,3.0
3617,0.0,4.0,0.0,4.0
3412,1.0,4.0,0.0,-1.0
3353,1.0,5.0,0.0,1.0
3153,2.0,5.0,0.0,-1.0
3015,2.0,6.0,0.0,1.0
2885,2.0,7.0,0.0,2.0


In [525]:
get_calculated_personal('Davi Ramos')

,Winner,R_fighter,Fighter,KD,Sig. str. %,Td %,Sub. att,Pass,Rev.,Sig. str. Hits,Sig. str. Attempts,Total str. Hits,Total str. Attempts,Td Hits,Td Attempts,Head Hits,Head Attempts,Body Hits,Body Attempts,Leg Hits,Leg Attempts,Distance Hits,Distance Attempts,Clinch Hits,Clinch Attempts,Ground Hits,Ground Attempts,DOB,Height,Reach,Stance,Weight,date,match_time,age,loss,win,draws,streak,cum_match_time,cum_KD,cum_Sub. att,cum_Pass,cum_Rev.,cum_Sig. str. Hits,cum_Sig. str. Attempts,cum_Total str. Hits,cum_Total str. Attempts,cum_Td Hits,cum_Td Attempts,cum_Head Hits,cum_Head Attempts,cum_Body Hits,cum_Body Attempts,cum_Leg Hits,cum_Leg Attempts,cum_Distance Hits,cum_Distance Attempts,cum_Clinch Hits,cum_Clinch Attempts,cum_Ground Hits,cum_Ground Attempts
1469,win,Sergio Moraes,Davi Ramos,0,0.28,0.00,0,0,0,35,122,35,122,0,1,21,97,7,17,7,8,35,122,0,0,0,0,05-11-1986,66,70,Orthodox,155,11-03-2017,15.00,30.37,0.0,0.0,0.0,0.0,15.00,0.0,0.0,0.0,0.0,35.0,122.0,35.0,122.0,0.0,1.0,21.0,97.0,7.0,17.0,7.0,8.0,35.0,122.0,0.0,0.0,0.0,0.0
1117,win,Davi Ramos,Davi Ramos,0,0.44,0.55,1,5,0,55,124,65,136,5,9,49,117,4,5,2,2,44,105,9,13,2,6,05-11-1986,66,70,Orthodox,155,09-12-2017,10.83,31.12,1.0,0.0,0.0,-1.0,15.00,0.0,0.0,0.0,0.0,35.0,122.0,35.0,122.0,0.0,1.0,21.0,97.0,7.0,17.0,7.0,8.0,35.0,122.0,0.0,0.0,0.0,0.0
942,win,Davi Ramos,Davi Ramos,0,0.50,1.00,1,2,0,6,12,13,21,1,1,5,9,1,3,0,0,5,11,0,0,1,1,05-11-1986,66,70,Orthodox,155,12-05-2018,4.25,31.54,1.0,1.0,0.0,1.0,25.83,0.0,1.0,5.0,0.0,90.0,246.0,100.0,258.0,5.0,10.0,70.0,214.0,11.0,22.0,9.0,10.0,79.0,227.0,9.0,13.0,2.0,6.0
706,win,Davi Ramos,Davi Ramos,0,0.80,1.00,1,2,0,4,5,4,5,1,1,3,4,1,1,0,0,0,1,0,0,4,4,05-11-1986,66,70,Orthodox,155,10-11-2018,1.95,32.04,1.0,2.0,0.0,2.0,30.08,0.0,2.0,7.0,0.0,96.0,258.0,113.0,279.0,6.0,11.0,75.0,223.0,12.0,25.0,9.0,10.0,84.0,238.0,9.0,13.0,3.0,7.0
418,win,Davi Ramos,Davi Ramos,0,0.55,0.42,0,3,0,63,114,76,128,3,7,44,86,7,15,12,13,41,92,5,5,17,17,05-11-1986,66,70,Orthodox,155,18-05-2019,15.00,32.55,1.0,3.0,0.0,3.0,32.03,0.0,3.0,9.0,0.0,100.0,263.0,117.0,284.0,7.0,12.0,78.0,227.0,13.0,26.0,9.0,10.0,84.0,239.0,9.0,13.0,7.0,11.0
258,win,Islam Makhachev,Davi Ramos,0,0.12,0.00,0,0,0,7,58,27,80,0,1,5,43,1,14,1,1,5,56,2,2,0,0,05-11-1986,66,70,Orthodox,155,07-09-2019,15.00,32.86,1.0,4.0,0.0,4.0,47.03,0.0,3.0,12.0,0.0,163.0,377.0,193.0,412.0,10.0,19.0,122.0,313.0,20.0,41.0,21.0,23.0,125.0,331.0,14.0,18.0,24.0,28.0


In [526]:
test = test_personal

In [528]:
test_wl

2974    False
2752     True
2641     True
2456     True
2241     True
2056    False
1910     True
1822     True
1632     True
1468     True
1340     True
1215    False
948      True
624     False
413     False
162      True
0       False
Name: R_fighter, dtype: bool

# Generate the Personals:

In [529]:
#personal_dict: key = name, values = personal_dfs with cumulative statistics

personal_dict = {}

for name in fighters.Name:
    print(name)
    personal_dict[name] = get_calculated_personal(name)

Gabriel Silva
Aalon Cruz
Davi Ramos
Sean McCorkle
Glaico Franca
Dylan Andrews
Luke Rockhold
John Salter
Joe Brammer
Gina Mazany
Rachael Ostovich
Chase Sherman
Benji Radach
Caros Fodor
Koji Oishi
Josh Bryant
Henry Briones
Jason Reinhardt
Darko Stosic
Sean Sherk
Abubakar Nurmagomedov
Dustin Poirier
Rudyard Moncayo
Kevin Jordan
Artem Lobov
Trent Jenkins
Kerry Schall
Khabib Nurmagomedov
Pete Sell
Bas Rutten
John Cholish
Walt Harris
Nate Schroeder
Arnold Allen
Jack Marshman
Marcus Aurelio
Matt Horwich
Oluwale Bamgbose
Nate Mohr
Tanner Boser
Jorge Gurgel
Gerald Strebendt
Chase Gormley
Rob Font
Houston Alexander
Rob Kimmons
Costas Philippou
Henry Cejudo
Jon Olav Einemo
Marvin Eastman
Dale Hartt
Chris Price
Kailin Curran
Andrea Lee
Mike Jackson
Cain Carrizosa
Devin Powell
Scott Askham
Maurice Greene
Branden Lee Hinkle
Ed Herman
Charles Oliveira
Milana Dudieva
Tarec Saffiedine
Joey Gomez
Paul Varelans
Rogerio Bontorin
Jon Jones
Volkan Oezdemir
Tommy Hayden
Dave Galera
Rafael Carino
Kimo Leopold

Jason Brilz
Jackie Lee
Nordine Taleb
Neil Seery
Eddie Mendez
Tyson Pedro
Icho Larenas
David Michaud
Cesar Arzamendia
John Lewis
Maximo Blanco
Stephen Thompson
Randa Markos
Jason Saggo
Alexis Dufresne
Alberto Mina
Anthony Perosh
Tim Means
James Zikic
Hector Urbina
Zak Cummings
Vitor Miranda
Chas Skelly
Domingo Pilarte
Luke Jumeau
Alexis Davis
Ryan LaFlare
Mikey Burnett
Dan Lauzon
Shane Young
Tom Lawlor
Paige VanZant
Brad Gumm
Jalin Turner
Harry Moskowitz
Wellington Turman
Krzysztof Jotko
Lyman Good
Loma Lookboonmee
Wesley Correira
Anthony Christodoulou
Herbert Burns
Jake Rosholt
Diego Saraiva
Ross Pearson
Carlos Eduardo Rocha
Daiju Takase
Carlo Prater
David Heath
Izabela Badurek
Sam Stout
Thaddeus Luster
Ricco Rodriguez
Pat Healy
Ovince Saint Preux
Molly McCann
Shane Primm
Michael Johnson
Brock Lesnar
Daichi Abe
Drew Dober
Liz Carmouche
Mike Ciesnolevicz
Mackens Semerzier
Thibault Gouti
Allan Zuniga
Hans Stringer
Court McGee
Luana Carolina
Johnny Eduardo
Elias Silverio
Youssef Zalal
Nat

Jesse Forbes
Felipe Olivieri
Carlos Newton
Giga Chikadze
Luigi Fioravanti
Jutaro Nakao
Robert Lucarelli
Robert Peralta
Luke Sanders
Polyana Viana
Frankie Perez
Onassis Parungao
Teila Tuli
Delson Heleno
Chase Hooper
Scott Ferrozzo
CJ Fernandes
Francisco Trevino
Jeff Hougland
Steve Garcia
Dave Strasser
Julie Kedzie
Cynthia Calvillo
James Bochnovic
Dong Hyun Ma
Kyoji Horiguchi
Matt Hamill
Aleksei Oleinik
Shane Burgos
Boston Salmon
Besam Yousef
Charlie Brenneman
Tim Williams
Andy Enz
John Dodson
Jordan Johnson
Grant Dawson
Erik Koch
Cathal Pendred
Genki Sudo
Jan Blachowicz
Tae Hyun Bang
Joao Pierini
Cory Hendricks
David Dvorak
Justin Salas
Ricky Rainey
Kazushi Sakuraba
Todd Brown
Paul Taylor
Justin Jones
Rustam Khabilov
Vanessa Melo
Frankie Saenz
Jermaine Andre
Dong Hyun Kim
Cyril Asker
Kelly Faszholz
Martin Bravo
Leonardo Santos
Greg Soto
Jimmie Rivera
Tim McKenzie
Mark Weir
Luan Chagas
Macy Chiasson
Darren Till
Geraldo de Freitas
Nicolas Dalby
Kuniyoshi Hironaka
Roman Kopylov
Antonio Ban

Cal Worsham
Christos Giagos
Matt Arroyo
Sergio Pettis
Thiago Santos
Anthony Macias
Brian Ortega
Chris Tuchscherer
Scott Holtzman
Mario Miranda
Noe Hernandez
Alex Stiebling
Joe Moreira
Chris Dempsey
Jamall Emmers
Steve Bosse
Yoji Anjo
Assuerio Silva
Curtis Stout
Mike Brown
Hermes Franca
Wes Sims
Isabela de Padua
Robert Whittaker
Alexander Volkanovski
Justin Tafa
Abel Trujillo
Bartosz Fabinski
Alptekin Ozkilic
Reza Madadi
Mike Lullo
Shamar Bailey
Aleksei Kunchenko
Serghei Spivac
Alex Karalexis
Marcos Vinicius
Keith Berish
Dustin Jacoby
Jason Gonzalez
Brian Gassaway
Rolles Gracie
Nick Denis
Sanae Kikuta
Mehdi Baghdad
Alex Torres
Paul Felder
Dustin Neace
Aisling Daly
Patrick Cummins
Chris de la Rocha
Michel Prazeres
Justin Edwards
Homer Moore
Jake Hecht
Luis Henrique
Yuki Sasaki
Evan Dunham
Tina Lahdemaki
Jason High
Chris Clements
Donny Walker
Allan Goes
Jingliang Li
Sherman Pendergarst
Lavar Johnson
Sultan Aliev
Sean Alvarez
Alexandra Albu
John Dowdy
Justin Wren
Casey Kenney
Adrian Serran

In [531]:
personal_df = pd.DataFrame()

for item in personal_dict.values():
    personal_df = pd.concat([personal_df, item], axis = 0)
    print(personal_df)

    Winner       R_fighter        Fighter  ...  cum_Clinch Attempts  cum_Ground Hits  cum_Ground Attempts
338    win        Ray Borg  Gabriel Silva  ...                  4.0              0.0                  0.0
28     win  Kyler Phillips  Gabriel Silva  ...                  4.0              0.0                  0.0

[2 rows x 62 columns]
    Winner       R_fighter        Fighter  ...  cum_Clinch Attempts  cum_Ground Hits  cum_Ground Attempts
338    win        Ray Borg  Gabriel Silva  ...                  4.0              0.0                  0.0
28     win  Kyler Phillips  Gabriel Silva  ...                  4.0              0.0                  0.0
33     win   Spike Carlyle     Aalon Cruz  ...                  0.0              0.0                  0.0

[3 rows x 62 columns]
     Winner        R_fighter        Fighter  ...  cum_Clinch Attempts  cum_Ground Hits  cum_Ground Attempts
338     win         Ray Borg  Gabriel Silva  ...                  4.0              0.0                  

     Winner            R_fighter        Fighter  ...  cum_Clinch Attempts  cum_Ground Hits  cum_Ground Attempts
338     win             Ray Borg  Gabriel Silva  ...                  4.0              0.0                  0.0
28      win       Kyler Phillips  Gabriel Silva  ...                  4.0              0.0                  0.0
33      win        Spike Carlyle     Aalon Cruz  ...                  0.0              0.0                  0.0
1469    win        Sergio Moraes     Davi Ramos  ...                  0.0              0.0                  0.0
1117    win           Davi Ramos     Davi Ramos  ...                  0.0              0.0                  0.0
942     win           Davi Ramos     Davi Ramos  ...                 13.0              2.0                  6.0
706     win           Davi Ramos     Davi Ramos  ...                 13.0              3.0                  7.0
418     win           Davi Ramos     Davi Ramos  ...                 13.0              7.0              

     Winner            R_fighter           Fighter  ...  cum_Clinch Attempts  cum_Ground Hits  cum_Ground Attempts
338     win             Ray Borg     Gabriel Silva  ...                  4.0              0.0                  0.0
28      win       Kyler Phillips     Gabriel Silva  ...                  4.0              0.0                  0.0
33      win        Spike Carlyle        Aalon Cruz  ...                  0.0              0.0                  0.0
1469    win        Sergio Moraes        Davi Ramos  ...                  0.0              0.0                  0.0
1117    win           Davi Ramos        Davi Ramos  ...                  0.0              0.0                  0.0
942     win           Davi Ramos        Davi Ramos  ...                 13.0              2.0                  6.0
706     win           Davi Ramos        Davi Ramos  ...                 13.0              3.0                  7.0
418     win           Davi Ramos        Davi Ramos  ...                 13.0    

     Winner            R_fighter           Fighter  ...  cum_Clinch Attempts  cum_Ground Hits  cum_Ground Attempts
338     win             Ray Borg     Gabriel Silva  ...                  4.0              0.0                  0.0
28      win       Kyler Phillips     Gabriel Silva  ...                  4.0              0.0                  0.0
33      win        Spike Carlyle        Aalon Cruz  ...                  0.0              0.0                  0.0
1469    win        Sergio Moraes        Davi Ramos  ...                  0.0              0.0                  0.0
1117    win           Davi Ramos        Davi Ramos  ...                  0.0              0.0                  0.0
942     win           Davi Ramos        Davi Ramos  ...                 13.0              2.0                  6.0
706     win           Davi Ramos        Davi Ramos  ...                 13.0              3.0                  7.0
418     win           Davi Ramos        Davi Ramos  ...                 13.0    

[68 rows x 62 columns]
     Winner                 R_fighter           Fighter  ...  cum_Clinch Attempts  cum_Ground Hits  cum_Ground Attempts
338     win                  Ray Borg     Gabriel Silva  ...                  4.0              0.0                  0.0
28      win            Kyler Phillips     Gabriel Silva  ...                  4.0              0.0                  0.0
33      win             Spike Carlyle        Aalon Cruz  ...                  0.0              0.0                  0.0
1469    win             Sergio Moraes        Davi Ramos  ...                  0.0              0.0                  0.0
1117    win                Davi Ramos        Davi Ramos  ...                  0.0              0.0                  0.0
942     win                Davi Ramos        Davi Ramos  ...                 13.0              2.0                  6.0
706     win                Davi Ramos        Davi Ramos  ...                 13.0              3.0                  7.0
418     win      

...

     Winner         R_fighter        Fighter  ...  cum_Clinch Attempts  cum_Ground Hits  cum_Ground Attempts
338     win          Ray Borg  Gabriel Silva  ...                  4.0              0.0                  0.0
28      win    Kyler Phillips  Gabriel Silva  ...                  4.0              0.0                  0.0
33      win     Spike Carlyle     Aalon Cruz  ...                  0.0              0.0                  0.0
1469    win     Sergio Moraes     Davi Ramos  ...                  0.0              0.0                  0.0
1117    win        Davi Ramos     Davi Ramos  ...                  0.0              0.0                  0.0
...     ...               ...            ...  ...                  ...              ...                  ...
2586    win   Colby Covington   Wagner Silva  ...                  0.0              0.0                  0.0
3322    win      Patrick Cote  Bobby Voelker  ...                 41.0             15.0                 25.0
3200    win     Rob

     Winner        R_fighter         Fighter  ...  cum_Clinch Attempts  cum_Ground Hits  cum_Ground Attempts
338     win         Ray Borg   Gabriel Silva  ...                  4.0              0.0                  0.0
28      win   Kyler Phillips   Gabriel Silva  ...                  4.0              0.0                  0.0
33      win    Spike Carlyle      Aalon Cruz  ...                  0.0              0.0                  0.0
1469    win    Sergio Moraes      Davi Ramos  ...                  0.0              0.0                  0.0
1117    win       Davi Ramos      Davi Ramos  ...                  0.0              0.0                  0.0
...     ...              ...             ...  ...                  ...              ...                  ...
1781    win   Eddie Wineland  Eddie Wineland  ...                 27.0             31.0                 55.0
1562    win   Eddie Wineland  Eddie Wineland  ...                 29.0             34.0                 59.0
1415    win      Jo

     Winner       R_fighter         Fighter  ...  cum_Clinch Attempts  cum_Ground Hits  cum_Ground Attempts
338     win        Ray Borg   Gabriel Silva  ...                  4.0              0.0                  0.0
28      win  Kyler Phillips   Gabriel Silva  ...                  4.0              0.0                  0.0
33      win   Spike Carlyle      Aalon Cruz  ...                  0.0              0.0                  0.0
1469    win   Sergio Moraes      Davi Ramos  ...                  0.0              0.0                  0.0
1117    win      Davi Ramos      Davi Ramos  ...                  0.0              0.0                  0.0
...     ...             ...             ...  ...                  ...              ...                  ...
3219    win  Brian Melancon  Seth Baczynski  ...                 69.0             26.0                 44.0
3088    win  Seth Baczynski  Seth Baczynski  ...                 71.0             26.0                 44.0
2873    win    Thiago Alves 

     Winner       R_fighter        Fighter  ...  cum_Clinch Attempts  cum_Ground Hits  cum_Ground Attempts
338     win        Ray Borg  Gabriel Silva  ...                  4.0              0.0                  0.0
28      win  Kyler Phillips  Gabriel Silva  ...                  4.0              0.0                  0.0
33      win   Spike Carlyle     Aalon Cruz  ...                  0.0              0.0                  0.0
1469    win   Sergio Moraes     Davi Ramos  ...                  0.0              0.0                  0.0
1117    win      Davi Ramos     Davi Ramos  ...                  0.0              0.0                  0.0
...     ...             ...            ...  ...                  ...              ...                  ...
2900    win       Jim Alers      Jim Alers  ...                 22.0             10.0                 14.0
2459    win     Chas Skelly      Jim Alers  ...                 22.0             10.0                 14.0
2044     nc     Cole Miller      Jim 

     Winner        R_fighter          Fighter  ...  cum_Clinch Attempts  cum_Ground Hits  cum_Ground Attempts
338     win         Ray Borg    Gabriel Silva  ...                  4.0              0.0                  0.0
28      win   Kyler Phillips    Gabriel Silva  ...                  4.0              0.0                  0.0
33      win    Spike Carlyle       Aalon Cruz  ...                  0.0              0.0                  0.0
1469    win    Sergio Moraes       Davi Ramos  ...                  0.0              0.0                  0.0
1117    win       Davi Ramos       Davi Ramos  ...                  0.0              0.0                  0.0
...     ...              ...              ...  ...                  ...              ...                  ...
3936    win  Rampage Jackson  Rampage Jackson  ...                140.0             46.0                 66.0
3834    win        Jon Jones  Rampage Jackson  ...                174.0             46.0                 66.0
3677    wi

# Calculate Efficiency:

In [534]:
cumulative_columns = ['cum_KD', 'cum_Sub. att', 'cum_Pass', 'cum_Rev.',
       'cum_Sig. str. Hits', 'cum_Sig. str. Attempts', 'cum_Total str. Hits',
       'cum_Total str. Attempts', 'cum_Td Hits', 'cum_Td Attempts',
       'cum_Head Hits', 'cum_Head Attempts', 'cum_Body Hits',
       'cum_Body Attempts', 'cum_Leg Hits', 'cum_Leg Attempts',
       'cum_Distance Hits', 'cum_Distance Attempts', 'cum_Clinch Hits',
       'cum_Clinch Attempts', 'cum_Ground Hits', 'cum_Ground Attempts']

In [536]:
for item in cumulative_columns:
    a = round(personal_df[item] / personal_df['cum_match_time'] * 10,2)
    personal_df['avg_{}'.format(item)] = a

In [613]:
personal_df.to_csv('personal_fights.csv')

# Finalize:

In [5]:
final_df = pd.read_csv('Data/personal_fights.csv', index_col = 0)
final_df.head(5)

,Winner,R_fighter,Fighter,KD,Sig. str. %,Td %,Sub. att,Pass,Rev.,Sig. str. Hits,...,avg_cum_Body Hits,avg_cum_Body Attempts,avg_cum_Leg Hits,avg_cum_Leg Attempts,avg_cum_Distance Hits,avg_cum_Distance Attempts,avg_cum_Clinch Hits,avg_cum_Clinch Attempts,avg_cum_Ground Hits,avg_cum_Ground Attempts
338,win,Ray Borg,Gabriel Silva,0,0.50,0.16,0,1,1,8,...,0.00,0.67,0.67,0.67,4.00,8.00,1.33,2.67,0.0,0.0
28,win,Kyler Phillips,Gabriel Silva,0,0.31,0.33,1,0,0,31,...,0.00,0.67,0.67,0.67,4.00,8.00,1.33,2.67,0.0,0.0
33,win,Spike Carlyle,Aalon Cruz,0,0.16,0.00,0,0,0,2,...,0.00,28.17,0.00,28.17,14.08,84.51,0.00,0.00,0.0,0.0
1469,win,Sergio Moraes,Davi Ramos,0,0.28,0.00,0,0,0,35,...,4.67,11.33,4.67,5.33,23.33,81.33,0.00,0.00,0.0,0.0
1117,win,Davi Ramos,Davi Ramos,0,0.44,0.55,1,5,0,55,...,4.67,11.33,4.67,5.33,23.33,81.33,0.00,0.00,0.0,0.0


In [6]:
final_df.columns

Index(['Winner', 'R_fighter', 'Fighter', 'KD', 'Sig. str. %', 'Td %',
       'Sub. att', 'Pass', 'Rev.', 'Sig. str. Hits', 'Sig. str. Attempts',
       'Total str. Hits', 'Total str. Attempts', 'Td Hits', 'Td Attempts',
       'Head Hits', 'Head Attempts', 'Body Hits', 'Body Attempts', 'Leg Hits',
       'Leg Attempts', 'Distance Hits', 'Distance Attempts', 'Clinch Hits',
       'Clinch Attempts', 'Ground Hits', 'Ground Attempts', 'DOB', 'Height',
       'Reach', 'Stance', 'Weight', 'date', 'match_time', 'age', 'loss', 'win',
       'draws', 'streak', 'cum_match_time', 'cum_KD', 'cum_Sub. att',
       'cum_Pass', 'cum_Rev.', 'cum_Sig. str. Hits', 'cum_Sig. str. Attempts',
       'cum_Total str. Hits', 'cum_Total str. Attempts', 'cum_Td Hits',
       'cum_Td Attempts', 'cum_Head Hits', 'cum_Head Attempts',
       'cum_Body Hits', 'cum_Body Attempts', 'cum_Leg Hits',
       'cum_Leg Attempts', 'cum_Distance Hits', 'cum_Distance Attempts',
       'cum_Clinch Hits', 'cum_Clinch Attempts', 

In [7]:
drop_columns = ['KD', 'Sig. str. %', 'Td %',
       'Sub. att', 'Pass', 'Rev.', 'Sig. str. Hits', 'Sig. str. Attempts',
       'Total str. Hits', 'Total str. Attempts', 'Td Hits', 'Td Attempts',
       'Head Hits', 'Head Attempts', 'Body Hits', 'Body Attempts', 'Leg Hits',
       'Leg Attempts', 'Distance Hits', 'Distance Attempts', 'Clinch Hits',
       'Clinch Attempts', 'Ground Hits', 'Ground Attempts']

In [8]:
final_df.drop(drop_columns, axis = 1, inplace = True)

In [13]:
final_df.columns

Index(['Winner', 'R_fighter', 'Fighter', 'DOB', 'Height', 'Reach', 'Stance',
       'Weight', 'date', 'match_time', 'age', 'loss', 'win', 'draws', 'streak',
       'cum_match_time', 'cum_KD', 'cum_Sub. att', 'cum_Pass', 'cum_Rev.',
       'cum_Sig. str. Hits', 'cum_Sig. str. Attempts', 'cum_Total str. Hits',
       'cum_Total str. Attempts', 'cum_Td Hits', 'cum_Td Attempts',
       'cum_Head Hits', 'cum_Head Attempts', 'cum_Body Hits',
       'cum_Body Attempts', 'cum_Leg Hits', 'cum_Leg Attempts',
       'cum_Distance Hits', 'cum_Distance Attempts', 'cum_Clinch Hits',
       'cum_Clinch Attempts', 'cum_Ground Hits', 'cum_Ground Attempts',
       'avg_cum_KD', 'avg_cum_Sub. att', 'avg_cum_Pass', 'avg_cum_Rev.',
       'avg_cum_Sig. str. Hits', 'avg_cum_Sig. str. Attempts',
       'avg_cum_Total str. Hits', 'avg_cum_Total str. Attempts',
       'avg_cum_Td Hits', 'avg_cum_Td Attempts', 'avg_cum_Head Hits',
       'avg_cum_Head Attempts', 'avg_cum_Body Hits', 'avg_cum_Body Attempts',
   

# Calculate fighter efficiency:

In [47]:
sig_str = final_df['cum_Sig. str. Hits'] / final_df['cum_Sig. str. Attempts']
hits = final_df['cum_Total str. Hits'] / final_df['cum_Total str. Attempts']
tds = final_df['cum_Td Hits'] / final_df['cum_Td Attempts']
head = final_df['cum_Head Hits'] / final_df['cum_Head Attempts']
leg = final_df['cum_Leg Hits'] / final_df['cum_Leg Attempts']
body = final_df['cum_Body Hits'] / final_df['cum_Body Attempts']
distance = final_df['cum_Distance Hits'] / final_df['cum_Distance Attempts']
clinch = final_df['cum_Clinch Hits'] / final_df['cum_Clinch Attempts']
ground = final_df['cum_Ground Hits'] / final_df['cum_Ground Attempts']

efficiency_stats = [sig_str, hits, tds, head, leg, body, distance, clinch, ground]
efficiency_columns = ['eff_sig_str', 'eff_hits', 'eff_tds', 'eff_head', 'eff_leg', 'eff_body', 
                      'eff_distance', 'eff_clinch', 'eff_ground']

for item in efficiency_stats:
    item.fillna(0, inplace = True)

In [53]:
for name, item in zip(efficiency_columns, efficiency_stats):
    final_df[name] = item
    final_df[name] = final_df[name].apply(lambda x: round(x, 2))


In [55]:
final_df.columns

Index(['Winner', 'R_fighter', 'Fighter', 'DOB', 'Height', 'Reach', 'Stance',
       'Weight', 'date', 'match_time', 'age', 'loss', 'win', 'draws', 'streak',
       'cum_match_time', 'cum_KD', 'cum_Sub. att', 'cum_Pass', 'cum_Rev.',
       'cum_Sig. str. Hits', 'cum_Sig. str. Attempts', 'cum_Total str. Hits',
       'cum_Total str. Attempts', 'cum_Td Hits', 'cum_Td Attempts',
       'cum_Head Hits', 'cum_Head Attempts', 'cum_Body Hits',
       'cum_Body Attempts', 'cum_Leg Hits', 'cum_Leg Attempts',
       'cum_Distance Hits', 'cum_Distance Attempts', 'cum_Clinch Hits',
       'cum_Clinch Attempts', 'cum_Ground Hits', 'cum_Ground Attempts',
       'avg_cum_KD', 'avg_cum_Sub. att', 'avg_cum_Pass', 'avg_cum_Rev.',
       'avg_cum_Sig. str. Hits', 'avg_cum_Sig. str. Attempts',
       'avg_cum_Total str. Hits', 'avg_cum_Total str. Attempts',
       'avg_cum_Td Hits', 'avg_cum_Td Attempts', 'avg_cum_Head Hits',
       'avg_cum_Head Attempts', 'avg_cum_Body Hits', 'avg_cum_Body Attempts',
   

In [56]:
drop_columns = ['cum_KD', 'cum_Sub. att', 'cum_Pass', 'cum_Rev.',
       'cum_Sig. str. Hits', 'cum_Sig. str. Attempts', 'cum_Total str. Hits',
       'cum_Total str. Attempts', 'cum_Td Hits', 'cum_Td Attempts',
       'cum_Head Hits', 'cum_Head Attempts', 'cum_Body Hits',
       'cum_Body Attempts', 'cum_Leg Hits', 'cum_Leg Attempts',
       'cum_Distance Hits', 'cum_Distance Attempts', 'cum_Clinch Hits',
       'cum_Clinch Attempts', 'cum_Ground Hits', 'cum_Ground Attempts']

In [57]:
final_df.drop(drop_columns, axis = 1, inplace = True)

In [58]:
final_df.columns

Index(['Winner', 'R_fighter', 'Fighter', 'DOB', 'Height', 'Reach', 'Stance',
       'Weight', 'date', 'match_time', 'age', 'loss', 'win', 'draws', 'streak',
       'cum_match_time', 'avg_cum_KD', 'avg_cum_Sub. att', 'avg_cum_Pass',
       'avg_cum_Rev.', 'avg_cum_Sig. str. Hits', 'avg_cum_Sig. str. Attempts',
       'avg_cum_Total str. Hits', 'avg_cum_Total str. Attempts',
       'avg_cum_Td Hits', 'avg_cum_Td Attempts', 'avg_cum_Head Hits',
       'avg_cum_Head Attempts', 'avg_cum_Body Hits', 'avg_cum_Body Attempts',
       'avg_cum_Leg Hits', 'avg_cum_Leg Attempts', 'avg_cum_Distance Hits',
       'avg_cum_Distance Attempts', 'avg_cum_Clinch Hits',
       'avg_cum_Clinch Attempts', 'avg_cum_Ground Hits',
       'avg_cum_Ground Attempts', 'eff_sig_str', 'eff_hits', 'eff_tds',
       'eff_head', 'eff_leg', 'eff_body', 'eff_distance', 'eff_clinch',
       'eff_ground'],
      dtype='object')

In [59]:
final_columns = ['Winner', 'R_fighter', 'Fighter', 'DOB', 'Height', 'Reach', 'Stance',
       'Weight', 'date', 'match_time', 'age', 'loss', 'win', 'draws', 'streak',
       'cum_match_time', 'avg_cum_KD', 'avg_cum_Sub. att', 'avg_cum_Pass',
       'avg_cum_Rev.', 'avg_cum_Sig. str. Hits', 'avg_cum_Sig. str. Attempts',
       'avg_cum_Total str. Hits', 'avg_cum_Total str. Attempts',
       'avg_cum_Td Hits', 'avg_cum_Td Attempts', 'avg_cum_Head Hits',
       'avg_cum_Head Attempts', 'avg_cum_Body Hits', 'avg_cum_Body Attempts',
       'avg_cum_Leg Hits', 'avg_cum_Leg Attempts', 'avg_cum_Distance Hits',
       'avg_cum_Distance Attempts', 'avg_cum_Clinch Hits',
       'avg_cum_Clinch Attempts', 'avg_cum_Ground Hits',
       'avg_cum_Ground Attempts', 'eff_sig_str', 'eff_hits', 'eff_tds',
       'eff_head', 'eff_leg', 'eff_body', 'eff_distance', 'eff_clinch',
       'eff_ground']

final_columns_1 = ['{}.1'.format(item) for item in final_columns]

In [60]:
pd.set_option('display.max_rows', 100)

reformat_final_df = pd.DataFrame()

for item in set(final_df.index):
    temp = pd.DataFrame(final_df.loc[item].iloc[0, :]).T
    temp1 = pd.DataFrame(final_df.loc[item].iloc[1, :]).T

    temp.columns = final_columns
    temp1.columns = final_columns_1
    
    combined = pd.concat([temp, temp1], axis = 1)
    reformat_final_df = pd.concat([reformat_final_df, combined], axis = 0)

In [61]:
reformat_final_df.to_csv('unformatted_final.csv')

# Format final df:

In [62]:
reformat_final_df = pd.read_csv('unformatted_final.csv', index_col = 0)
reformat_final_df.head(5)

,Winner,R_fighter,Fighter,DOB,Height,Reach,Stance,Weight,date,match_time,...,avg_cum_Ground Attempts.1,eff_sig_str.1,eff_hits.1,eff_tds.1,eff_head.1,eff_leg.1,eff_body.1,eff_distance.1,eff_clinch.1,eff_ground.1
0,win,Charles Oliveira,Charles Oliveira,17-10-1989,61,74,Orthodox,155,14-03-2020,10.47,...,17.93,0.43,0.51,0.43,0.38,1.00,0.75,0.31,0.74,0.76
1,win,Gilbert Burns,Gilbert Burns,20-07-1986,61,71,Orthodox,170,14-03-2020,2.57,...,9.51,0.44,0.61,0.26,0.40,0.78,0.71,0.31,0.66,0.75
2,win,Renato Moicano,Damir Hadzovic,08-08-1986,69,70,Orthodox,155,14-03-2020,0.73,...,2.45,0.47,0.48,0.55,0.38,0.85,0.59,0.46,0.60,0.68
3,win,Nikita Krylov,Nikita Krylov,07-03-1992,75,77,Orthodox,205,14-03-2020,15.00,...,24.39,0.73,0.73,0.00,0.68,1.00,1.00,0.73,1.00,0.67
4,win,Francisco Trinaldo,Francisco Trinaldo,24-08-1978,69,70,Southpaw,155,14-03-2020,15.00,...,3.01,0.51,0.52,0.00,0.42,0.78,0.58,0.51,0.50,0.49


In [63]:
drop_columns = ['Winner.1', 'R_fighter.1', 'match_time', 'match_time.1', 'DOB', 'DOB.1', 'date', 'date.1']
reformat_final_df.drop(drop_columns, axis = 1, inplace = True)

In [65]:
reformat_final_df.columns

Index(['Winner', 'R_fighter', 'Fighter', 'Height', 'Reach', 'Stance', 'Weight',
       'age', 'loss', 'win', 'draws', 'streak', 'cum_match_time', 'avg_cum_KD',
       'avg_cum_Sub. att', 'avg_cum_Pass', 'avg_cum_Rev.',
       'avg_cum_Sig. str. Hits', 'avg_cum_Sig. str. Attempts',
       'avg_cum_Total str. Hits', 'avg_cum_Total str. Attempts',
       'avg_cum_Td Hits', 'avg_cum_Td Attempts', 'avg_cum_Head Hits',
       'avg_cum_Head Attempts', 'avg_cum_Body Hits', 'avg_cum_Body Attempts',
       'avg_cum_Leg Hits', 'avg_cum_Leg Attempts', 'avg_cum_Distance Hits',
       'avg_cum_Distance Attempts', 'avg_cum_Clinch Hits',
       'avg_cum_Clinch Attempts', 'avg_cum_Ground Hits',
       'avg_cum_Ground Attempts', 'eff_sig_str', 'eff_hits', 'eff_tds',
       'eff_head', 'eff_leg', 'eff_body', 'eff_distance', 'eff_clinch',
       'eff_ground', 'Fighter.1', 'Height.1', 'Reach.1', 'Stance.1',
       'Weight.1', 'age.1', 'loss.1', 'win.1', 'draws.1', 'streak.1',
       'cum_match_time.1', 'av

# Recombine Relevant Data:

In [86]:
combined_df.columns
combined_df['num_rounds'] = combined_df['format'].apply(lambda x: int(x[0]) if x[0] != 'N' else 1)

In [87]:
keep_columns = ['WEIGHT_CLASS', 'title_bout', 'location', 'attendance', 'num_rounds']
event_attr = combined_df[keep_columns].copy()

In [88]:
reformat_final_df = pd.concat([reformat_final_df, event_attr], axis = 1)

In [89]:
# Label:

winner = []
for index, row in reformat_final_df.iterrows():
    
    if row['Winner'] != 'win':
        winner.append(99999)
    
    elif (row['R_fighter'] == row['Fighter']):
        winner.append(0)
    else:
        winner.append(1)
        
reformat_final_df['label'] = winner

In [93]:
reformat_final_df['matches'] = reformat_final_df['win'] + reformat_final_df['loss'] + reformat_final_df['draws']
reformat_final_df['matches.1'] = reformat_final_df['win.1'] + reformat_final_df['loss.1'] + reformat_final_df['draws.1']

In [94]:
reformat_final_df.to_csv('model_data.csv')